In [1]:
import pandas as pd
import numpy as np
import mysql.connector
import os
from mysql.connector import Error
from datetime import datetime, date, timedelta
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#Definimos la ruta en donde queremos guardar los archivos
mes2 = 'Enero2021'
ruta_guardar = '/home/estadistico/Documents/Erick/'+mes2+'/Reportes diarios/Avance pagos'
ruta_pagos = '/home/estadistico/Documents/Erick'
mes = 202010101
mes3 = pd.to_datetime('01/01/2021',dayfirst=True)
prim_sem_mes = 1

In [3]:
quetzal = 2.54
dolar = 19.8

In [4]:
#Asignamos valores a los parametros"
today = date.today().strftime('%Y%m%d')
now = datetime.now().strftime('%d-%m-%Y %H-%M')
hoy = np.datetime64('today')
ayer = np.datetime64(date.today() - timedelta(1))
today2 = date.today().strftime('%d-%m-%Y')

# Torreon

In [5]:
metas = pd.DataFrame([('Avon',990000),('Oriflame',65000),('Invex_RLN',270000),('Invex_RLA',479000),('Invex_REL',470000),('Arabela',95000),('Cristian Lay',6000)])
metas.columns = ['Cliente','Meta']

In [6]:
#Lectura pagos Oriflam
#pagos_oriflame = pd.read_excel(''+ruta_pagos+'/'+mes2+'/Torreon/Oriflame/RECUPERACION EXITOSA 31-12-2020.xlsx')
pagos_oriflame = pd.read_excel(''+ruta_pagos+'/'+mes2+'/Torreon/Oriflame/RECUPERACION EXITOSA 12-01-2021.xlsx')
pagos_oriflame['Cliente'] = 'Oriflame'
pagos_oriflame = pagos_oriflame.drop_duplicates()
pagos_oriflame['Factura'] = pagos_oriflame['PAGO_TOTAL']*.1
# pagos_oriflame = pagos_oriflame.loc[pagos_oriflame['FECHA_PAGO']>=mes3]

In [7]:
#Lectura pagos Arabela
# pagos_arabela = pd.read_excel(''+ruta_pagos+'/'+mes2+'/Torreon/Arabela/31122020_PAGOS_ARABELA.xlsx')
pagos_arabela = pd.read_excel(''+ruta_pagos+'/'+mes2+'/Torreon/Arabela/PAGOS_ARABELA_12012021.xlsx')
pagos_arabela['Cliente'] = 'Arabela'
pagos_arabela = pagos_arabela.drop_duplicates()
pagos_arabela['FechaEntrega'] = [datetime.strptime(str(x), '%Y%m%d').strftime('%d/%m/%Y') for x in pagos_arabela['FechaEntrega']]
pagos_arabela['FechaEntrega'] = pd.to_datetime(pagos_arabela['FechaEntrega'],dayfirst=True)
# pagos_arabela = pagos_arabela.loc[pagos_arabela['FechaEntrega']>=mes3]
pagos_arabela['factura'] = pagos_arabela['pago']*.22

In [8]:
#Lectura pagos invex
# pagos_invex = pd.read_excel(''+ruta_pagos+'/'+mes2+'/Torreon/Invex/Pagos Invex.xlsx')
pagos_invex = pd.read_excel(''+ruta_pagos+'/'+mes2+'/Torreon/Invex/Pagos Invex '+mes2+'.xlsx')
pagos_invex['Cliente'] = 'Invex_'+pagos_invex['Agencia'].map(str)
pagos_invex = pagos_invex.drop_duplicates()
# pagos_invex = pagos_invex.loc[pagos_invex['TRAN_POST_DT']>=mes3]

# Oriflame

In [9]:
reporte_pagos_oriflame = pd.pivot_table(pagos_oriflame,index=['Cliente'],values=['Factura'],aggfunc=np.sum)
reporte_pagos_oriflame = pd.DataFrame(reporte_pagos_oriflame.to_records())
reporte_pagos_oriflame.columns = ['Cliente','Facturacion']
reporte_pagos_oriflame['FechaFact'] = max(pagos_oriflame['FECHA_PAGO'])
reporte_pagos_oriflame['FechaFact'] = pd.to_datetime(reporte_pagos_oriflame['FechaFact'],dayfirst=True)
reporte_pagos_oriflame

,Cliente,Facturacion,FechaFact
0,Oriflame,34482.005,2021-01-11


# Invex

In [10]:
reporte_pagos_invex = pd.pivot_table(pagos_invex,index=['Cliente'],values=['AMT_TRAN','HONORARIO'],aggfunc=[np.sum,np.mean])
reporte_pagos_invex = pd.DataFrame(reporte_pagos_invex.to_records())
reporte_pagos_invex = reporte_pagos_invex.iloc[:,[0,1,4]]
reporte_pagos_invex.columns = ['Cliente','Pago','Honorario']
reporte_pagos_invex['Facturacion'] = reporte_pagos_invex['Pago']*reporte_pagos_invex['Honorario']
reporte_pagos_invex = reporte_pagos_invex.iloc[:,[0,3]]
reporte_pagos_invex['FechaFact'] = max(pagos_invex['TRAN_POST_DT'])
reporte_pagos_invex['FechaFact'] = pd.to_datetime(reporte_pagos_invex['FechaFact'],dayfirst=True)
reporte_pagos_invex

,Cliente,Facturacion,FechaFact
0,Invex_REL,67912.764,2021-01-11
1,Invex_RLA,78187.004,2021-01-11
2,Invex_RLN,58453.902,2021-01-11


# Arabela

In [11]:
reporte_pagos_arabela = pd.pivot_table(pagos_arabela,index=['Cliente'],values=['factura'],aggfunc=np.sum)
reporte_pagos_arabela = pd.DataFrame(reporte_pagos_arabela.to_records())
reporte_pagos_arabela.columns = ['Cliente','Facturacion']
reporte_pagos_arabela['FechaFact'] = max(pagos_arabela['FechaEntrega'])
reporte_pagos_arabela['FechaFact'] = pd.to_datetime(reporte_pagos_arabela['FechaFact'],dayfirst=True)
reporte_pagos_arabela

,Cliente,Facturacion,FechaFact
0,Arabela,47829.2936,2021-01-12


# Union sucursal

In [12]:
pdList = [reporte_pagos_arabela,reporte_pagos_invex,reporte_pagos_oriflame]
final_torreon = pd.concat(pdList,ignore_index=True)
final_torreon = pd.merge(final_torreon,metas,on=['Cliente'],how='left')
final_torreon['Avance%'] = final_torreon['Facturacion']/final_torreon['Meta']
final_torreon.insert(loc=0,column='Sucursal',value='Torreon')
final_torreon

,Sucursal,Cliente,Facturacion,FechaFact,Meta,Avance%
0,Torreon,Arabela,47829.2936,2021-01-12,95000,0.503466
1,Torreon,Invex_REL,67912.7640,2021-01-11,470000,0.144495
2,Torreon,Invex_RLA,78187.0040,2021-01-11,479000,0.163230
3,Torreon,Invex_RLN,58453.9020,2021-01-11,270000,0.216496
4,Torreon,Oriflame,34482.0050,2021-01-11,65000,0.530492


# Naucalpan

In [13]:
meta_naucalpan = pd.DataFrame([['Liverpool', 300000],['Liverpool Cof', 300000],['Liverpool Nacional', 550000],['Bradesco',750000],['Credifiel',100000]])
meta_naucalpan.columns = ['Cliente','Meta']

In [14]:
#Lectura pagos Bradesco
pagos_brad = pd.read_excel(''+ruta_pagos+'/'+str(mes2)+'/Bradesco/Pagos/Acumulado de pagos enero de 2021.xlsx')
pagos_brad = pagos_brad.iloc[:,0:9]
pagos_brad = pagos_brad.drop_duplicates()
pagos_brad2 = pd.pivot_table(pagos_brad,index=['NUMERO_TARJETA','FECHA_PAGO','TIPO_ASIGANCION'],values=['MONTO'],aggfunc=np.sum)
pagos_brad2 = pd.DataFrame(pagos_brad2.to_records())
pagos_brad2 = pagos_brad2.loc[pagos_brad2['FECHA_PAGO']>=mes3]

In [15]:
#Lectura pagos Liverpool
sheet_liv = ['SA002REM','SA001REM','DKN04REM','DKN02REM','DKN00REM']
pagos_liv = pd.DataFrame()
for i in sheet_liv:
    pagos_liv_new = pd.read_excel(''+ruta_pagos+'/'+str(mes2)+'/Liverpool/Pagos/PAGOS ENERO2021.xlsx',sheet_name = str(i))
    pagos_liv = pd.concat([pagos_liv,pagos_liv_new],ignore_index=True)
pagos_liv = pagos_liv.drop_duplicates()
pagos_liv = pagos_liv.iloc[:,0:6]
pagos_liv.columns = ['Id','Cuenta','Fecha Prom','Fecha Pago','Monto','Posteo']
pagos_liv['Fecha Pago'] = pd.to_datetime(pagos_liv['Fecha Pago'],dayfirst=True)
pagos_liv2 = pd.pivot_table(pagos_liv,index=['Id','Cuenta','Fecha Pago','Fecha Prom'],values=['Monto'],aggfunc=np.sum)
pagos_liv2 = pd.DataFrame(pagos_liv2.to_records())
pagos_liv2 = pagos_liv2.loc[pagos_liv2['Fecha Pago']>=mes3]

In [16]:
#Lectura pagos Credifiel
sheet_cred = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Credifiel/Pagos/')
pagos_cred = pd.DataFrame()
for i in sheet_cred:
    pagos_cred_new = pd.read_excel(''+ruta_pagos+'/'+str(mes2)+'/Credifiel/Pagos/'+i+'')
    pagos_cred = pd.concat([pagos_cred,pagos_cred_new],ignore_index=True)
pagos_cred = pagos_cred.iloc[:,0:5]
pagos_cred = pagos_cred.drop_duplicates()
pagos_cred2 = pd.pivot_table(pagos_cred,index=['ID','FECHA'],values=['IMPORTE'],aggfunc=np.sum)
pagos_cred2 = pd.DataFrame(pagos_cred2.to_records())

In [17]:
#Hacemos la consulta de la asignacion
asignacion_cred = pd.read_excel('/home/estadistico/Documents/Erick/Credifiel/Asignacion/Asignacion Credifiel '+mes2+'.xlsx')
asignacion_cred = asignacion_cred.drop_duplicates(subset=['CREDITO','SALDO TOTAL REAL'])
asignacion_cred['CREDITO'] = pd.to_numeric(asignacion_cred['CREDITO'])
asignacion_cred = asignacion_cred[['CREDITO','ARANCEL']]

In [18]:
# #Lectura pagos Banco Azteca Recovery
# sheet_baz_recovery = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/Recovery/')
# pagos_baz_recovery = pd.DataFrame()
# for k in sheet_baz_recovery:
#     pagos_baz_recovery_new = pd.read_csv(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/Recovery/'+k+'',sep='|')
#     pagos_baz_recovery = pd.concat([pagos_baz_recovery,pagos_baz_recovery_new],ignore_index=True)
# pagos_baz_recovery['Cartera'] = 'Recovery'
# pagos_baz_recovery['DIA'] = pd.to_datetime(pagos_baz_recovery['DIA'],dayfirst=True)
# pagos_baz_recovery = pagos_baz_recovery.loc[pagos_baz_recovery['DIA']>=mes3]
# pagos_baz_recovery = pagos_baz_recovery.loc[pagos_baz_recovery['DIA']>=mes3]
# pagos_baz_recovery['credito'] = pagos_baz_recovery['PAIS'].map(str)+'-'+pagos_baz_recovery['CANAL'].map(str)+'-'+pagos_baz_recovery['SUCURSAL'].map(str)+'-'+pagos_baz_recovery['FOLIO'].map(str)
# pagos_baz_recovery = pagos_baz_recovery.drop_duplicates()

In [19]:
# #Lectura pagos Banco Azteca 26-39
# sheet_baz_26 = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/26-39/')
# pagos_baz_26 = pd.DataFrame()
# for j in sheet_baz_26:
#     pagos_baz_26_new = pd.read_csv(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/26-39/'+j+'',sep='|')
#     pagos_baz_26 = pd.concat([pagos_baz_26,pagos_baz_26_new],ignore_index=True)
# pagos_baz_26['cartera'] = '26-39'
# pagos_baz_26['DIA'] = pd.to_datetime(pagos_baz_26['DIA'],dayfirst=True)
# pagos_baz_26 = pagos_baz_26.loc[pagos_baz_26['DIA']>=mes3]
# pagos_baz_26['credito'] = pagos_baz_26['PAIS'].map(str)+'-'+pagos_baz_26['CANAL'].map(str)+'-'+pagos_baz_26['SUCURSAL'].map(str)+'-'+pagos_baz_26['FOLIO'].map(str)
# pagos_baz_26 = pagos_baz_26.drop_duplicates()

# Liverpool

In [20]:
#Hacemos la asignacion de cada una de las metas para cambiar de arancel sin tomar el 0 en cuenta
cof, arancel_cof,arancel_cof2 = 300000, .5, .5
sub, arancel_sub, arancel_sub2 = 300000, .15, .2
subdic, arancel_subdic, arancel_subdic2 = 350000, .16, .2
n2, arancel_n2, arancel_n22 = 650000, .4, .5
pren, arancel_pren, arancel_pren2 = 300000, .28, .28

In [21]:
#Asignamos los aranceles de liverpool
aranceles = [('DKN00REM','Liverpool'),('DKN02REM','Liverpool Nacional'),('DKN04REM','Liverpool Cof'),('SA001REM','Liverpool'),('SA002REM','Liverpool')]
aranceles = pd.DataFrame(aranceles)
aranceles.columns = ['Id','Cliente']

In [22]:
#Obtenemos los pagos totales por division
pagos_totales_liv = pd.pivot_table(pagos_liv,index=['Id'],values=['Monto'],aggfunc=[np.sum])
pagos_totales_liv = pd.DataFrame(pagos_totales_liv.to_records())
#pagos_totales_liv = pagos_totales_liv2.iloc[:,[0,2,1]]
pagos_totales_liv.columns = ['Division','SumaPagos']
pagos_totales_liv.fillna(0,inplace=True)
pagos_totales_liv

,Division,SumaPagos
0,DKN00REM,302061.00
1,DKN02REM,270882.99
2,DKN04REM,143493.37
3,SA001REM,100220.98
4,SA002REM,4401.00


In [23]:
DKN00REM = pagos_totales_liv.loc[pagos_totales_liv['Division']=='DKN00REM', 'SumaPagos'].item()
DKN02REM = pagos_totales_liv.loc[pagos_totales_liv['Division']=='DKN02REM', 'SumaPagos'].item()
DKN04REM = pagos_totales_liv.loc[pagos_totales_liv['Division']=='DKN04REM', 'SumaPagos'].item()
SA001REM = pagos_totales_liv.loc[pagos_totales_liv['Division']=='SA001REM', 'SumaPagos'].item()
SA002REM = pagos_totales_liv.loc[pagos_totales_liv['Division']=='SA002REM', 'SumaPagos'].item()

In [24]:
def arancel_liv(row):
    if row['Id'] == 'DKN00REM' :
        return np.where((DKN00REM>0) & (DKN00REM<=pren/arancel_pren),arancel_pren,arancel_pren2)
    elif row['Id'] == 'DKN02REM':
        return np.where((DKN02REM>0) & (DKN02REM<=n2/arancel_n2),arancel_n2,arancel_n22)
    elif row['Id'] == 'DKN04REM':
        return np.where((DKN04REM>0) & (DKN04REM<=cof/arancel_cof),arancel_cof,arancel_cof2)
    elif row['Id'] == 'SA001REM':
        return np.where((SA001REM>0) & (SA001REM<=sub/arancel_sub),arancel_sub,arancel_sub2)
    elif row['Id'] == 'SA002REM':
        return np.where((SA002REM>0) & (SA002REM<=subdic/arancel_subdic),arancel_subdic,arancel_subdic2)
    else:
        return 0


In [25]:
pagos_liv['Arancel'] = pagos_liv.apply(arancel_liv, axis=1)

#Agregamos aranceles al consolidado
pagos_liv2 = pd.merge(pagos_liv,aranceles,how='left',on='Id')
pagos_liv2['PagoArancel'] = [x*y for x,y in zip(pagos_liv2['Monto'],pagos_liv2['Arancel'])]
pagos_liv2['Fecha Pago'] = pd.to_datetime(pagos_liv2['Fecha Pago'],dayfirst=True)
pagos_liv2.head(1)

,Id,Cuenta,Fecha Prom,Fecha Pago,Monto,Posteo,Arancel,Cliente,PagoArancel
0,SA002REM,4050670000337528,07/01/2021,2021-01-07,500.0,5200000000009690,0.16,Liverpool,80.0


In [26]:
#Total pagos Liver registrados
pagos_totales_liv2 = pd.pivot_table(pagos_liv2,index=['Cliente'],values=['PagoArancel'],aggfunc=np.sum,dropna=False)
pagos_totales_liv2 = pd.DataFrame(pagos_totales_liv2.to_records())
#pagos_totales_liv = pagos_totales_liv.iloc[:,[0,3,1,2]]
pagos_totales_liv2.columns = ['Cliente','Facturacion']
pagos_totales_liv2.fillna(0,inplace=True)
pagos_totales_liv2['Facturacion'] = pagos_totales_liv2['Facturacion'].round(2)
pagos_totales_liv2['FechaFact'] = max(pagos_liv2['Fecha Pago'])
pagos_totales_liv2['FechaFact'] = pd.to_datetime(pagos_totales_liv2['FechaFact'],dayfirst=True)
pagos_totales_liv2

,Cliente,Facturacion,FechaFact
0,Liverpool,100314.39,2021-01-14
1,Liverpool Cof,71746.68,2021-01-14
2,Liverpool Nacional,108353.20,2021-01-14


In [27]:
def sig_arancel_liv(row):
    if row['Asignacion'] == 'DKN00REM' :
        return np.where((DKN00REM>0) & (DKN00REM<=pren/arancel_pren),round(pren/arancel_pren,1),0)
    elif row['Asignacion'] == 'DKN02REM':
        return np.where((DKN02REM>0) & (DKN02REM<=n2/arancel_n2),n2/arancel_n2,0)
    elif row['Asignacion'] == 'DKN04REM':
        return np.where((DKN04REM>0) & (DKN04REM<=cof/arancel_cof),cof/arancel_cof,0)
    elif row['Asignacion'] == 'SA001REM':
        return np.where((SA001REM>0) & (SA001REM<=sub/arancel_sub),sub/arancel_sub,0)
    elif row['Asignacion'] == 'SA002REM':
        return np.where((SA002REM>0) & (SA002REM<=subdic/arancel_subdic),subdic/arancel_subdic,0)
    
    else:
        return 0

In [28]:
pagos_liv_gerencia = pd.pivot_table(pagos_liv2,index=['Id'],values=['Monto'],aggfunc=[np.sum])
pagos_liv_gerencia = pd.DataFrame(pagos_liv_gerencia.to_records())
pagos_liv_gerencia.columns = ['Asignacion','Recuperado']
pagos_liv_gerencia.fillna(0,inplace=True)
pagos_liv_gerencia['FechaFact'] = max(pagos_liv2['Fecha Pago'])
pagos_liv_gerencia['SiguienteArancel'] = pagos_liv_gerencia.apply(sig_arancel_liv, axis=1)
pagos_liv_gerencia['RestSigArancel'] = pagos_liv_gerencia['SiguienteArancel']-pagos_liv_gerencia['Recuperado']
cols = ['Recuperado','SiguienteArancel','RestSigArancel']
pagos_liv_gerencia[cols] = pagos_liv_gerencia[cols].round(2)
pagos_liv_gerencia['RestSigArancel'] = np.where(pagos_liv_gerencia['RestSigArancel']<0,'Alcanzado',pagos_liv_gerencia['RestSigArancel'])
#pagos_liv_gerencia = pagos_liv_gerencia.iloc[:,[0,1,2,4]]
pagos_liv_gerencia.columns = ['Division','Recuperado','FechaFact','SiguienteArancel','RestSigArancel']
pagos_liv_gerencia

,Division,Recuperado,FechaFact,SiguienteArancel,RestSigArancel
0,DKN00REM,302061.00,2021-01-14,1071428.6,769368
1,DKN02REM,270882.99,2021-01-14,1625000.0,1.35412e+06
2,DKN04REM,143493.37,2021-01-14,600000.0,456507
3,SA001REM,100220.98,2021-01-14,2000000.0,1.89978e+06
4,SA002REM,4401.00,2021-01-14,2187500.0,2.1831e+06


# Bradesco

In [29]:
#Hacemos la asignacion de cada una de las metas para cambiar de arancel sin tomar el 0 en cuenta
c04pv1, arancel_4pv1 = 846436, .07
c04pv2, arancel_4pv2 = 1269654, .08
c04pv3, arancel_4pv3 = 1692873, .12
c05pv1, arancel_5pv1 = 729630, .09
c05pv2, arancel_5pv2 = 1094445, .10
c05pv3, arancel_5pv3 = 1459261, .14
c06pv1, arancel_6pv1 = 623524, .10
c06pv2, arancel_6pv2 = 935287, .11
c06pv3, arancel_6pv3 = 1247050, .16
a1, arancel_a1 = 264024, .27
a2, arancel_a2 = 396037, .30
a3, arancel_a3 = 528050, .33
b1, arancel_b1 = 214969, .25
b2, arancel_b2 = 396037, .28
b3, arancel_b3 = 528050, .31
c11, arancel_c11 = 1430989, .17
c12, arancel_c12 = 2146483, .20
c13, arancel_c13 = 2861978, .23
c21, arancel_c21 = 549549, .19
c22, arancel_c22 = 824323, .22
c23, arancel_c23 = 1099098, .25
c31, arancel_c31 = 237647, .21
c32, arancel_c32 = 356470, .24
c33, arancel_c33 = 475294, .27
c41, arancel_c41 = 109401, .23
c42, arancel_c42 = 164101, .26
c43, arancel_c43 = 218801, .29

In [30]:
#Obtenemos los pagos totales por division
pagos_totales_brad = pd.pivot_table(pagos_brad2,index=['TIPO_ASIGANCION'],values=['MONTO'],aggfunc=[np.sum,'count'],margins=True,dropna=False)
pagos_totales_brad = pd.DataFrame(pagos_totales_brad.to_records())
pagos_totales_brad = pagos_totales_brad.iloc[:,[0,2,1]]
pagos_totales_brad.columns = ['Division','NumeroPagos','SumaPagos']
pagos_totales_brad.fillna(0,inplace=True)

In [31]:
C04PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 4PV', 'SumaPagos'].item()
C05PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 5PV', 'SumaPagos'].item()
C06PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 6PV', 'SumaPagos'].item()
A = pagos_totales_brad.loc[pagos_totales_brad['Division']=='A', 'SumaPagos'].item()
C1 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C1', 'SumaPagos'].item()
C2 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C2', 'SumaPagos'].item()
C3 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C3', 'SumaPagos'].item()
C4 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C4', 'SumaPagos'].item()
B = pagos_totales_brad.loc[pagos_totales_brad['Division']=='B', 'SumaPagos'].item()

In [32]:
def sig_arancel(row):
    if row['Asignacion'] == 'C0 4PV':
        return np.where((C04PV>0) & (C04PV<=c04pv1),c04pv1,
                        np.where((C04PV>c04pv1) & (C04PV<=c04pv2),c04pv2,c04pv3))
    elif row['Asignacion'] == 'C0 5PV':
        return np.where((C05PV>0) & (C05PV<=c05pv1),c05pv1,
                        np.where((C05PV>c05pv1) & (C05PV<=c05pv2),c05pv2,c05pv3))
    elif row['Asignacion'] == 'C0 6PV':
        return np.where((C06PV>0) & (C06PV<=c06pv1),c06pv1,
                        np.where((C06PV>c06pv1) & (C06PV<=c06pv2),c06pv2,c06pv3))
    elif row['Asignacion'] == 'A':
        return np.where((A>0) & (A<=a1),a1,
                        np.where((A>a1) & (A<=a2),a2,a3))
    elif row['Asignacion'] == 'B':
        return np.where((B>0) & (B<=b1),b1,
                        np.where((B>b1) & (B<=b2),b2,b3))
    elif row['Asignacion'] == 'C1':
        return np.where((C1>0) & (C1<=c11),c11,
                        np.where((C1>c11) & (C1<=c12),c12,c13))
    elif row['Asignacion'] == 'C2':
        return np.where((C2>0) & (C2<=c21),c21,
                        np.where((C2>c21) & (C2<=c22),c22,c23))
    elif row['Asignacion'] == 'C3':
        return np.where((C3>0) & (C3<=c31),c31,
                        np.where((C3>c31) & (C3<=c32),c32,c33))
    elif row['Asignacion'] == 'C4':
        return np.where((C4>0) & (C4<=c41),c41,
                        np.where((C4>c41) & (C4<=c42),c42,c43))
    
    else:
        return 0

In [33]:
def func(row):
    if row['TIPO_ASIGANCION'] == 'C0 4PV':
        return np.where((C04PV>0) & (C04PV<=c04pv1),arancel_4pv1,
                        np.where((C04PV>c04pv1) & (C04PV<=c04pv2),arancel_4pv2,
                                 np.where((C04PV>c04pv2) & (C04PV<=c04pv3),arancel_4pv3,arancel_4pv3+.04)))
    elif row['TIPO_ASIGANCION'] == 'C0 5PV':
        return np.where((C05PV>0) & (C05PV<=c05pv1),arancel_5pv1,
                        np.where((C05PV>c05pv1) & (C05PV<=c05pv2),arancel_5pv2,
                                 np.where((C05PV>c05pv2) & (C05PV<=c05pv3),arancel_5pv3,arancel_5pv3+.04)))
    elif row['TIPO_ASIGANCION'] == 'C0 6PV':
        return np.where((C06PV>0) & (C06PV<=c06pv1),arancel_6pv1,
                        np.where((C06PV>c06pv1) & (C06PV<=c06pv2),arancel_6pv2,
                                 np.where((C06PV>c06pv2) & (C06PV<=c06pv3),arancel_6pv3,arancel_6pv3+.04)))
    elif row['TIPO_ASIGANCION'] == 'A':
        return np.where((A>0) & (A<=a1),arancel_a1,
                        np.where((A>a1) & (A<=a2),arancel_a2,
                                 np.where((A>a2) & (A<=a3),arancel_a3,arancel_a3+.04)))
    elif row['TIPO_ASIGANCION'] == 'B':
        return np.where((B>0) & (B<=b1),arancel_b1,
                        np.where((B>b1) & (B<=b2),arancel_b2,
                                 np.where((B>b2) & (B<=b3),arancel_b3,arancel_b3+.04)))
    elif row['TIPO_ASIGANCION'] == 'C1':
        return np.where((C1>0) & (C1<=c11),arancel_c11,
                        np.where((C1>c11) & (C1<=c12),arancel_c12,
                                 np.where((C1>c12) & (C1<=c13),arancel_c13,arancel_c13+.04)))
    elif row['TIPO_ASIGANCION'] == 'C2':
        return np.where((C2>0) & (C2<=c21),arancel_c21,
                        np.where((C2>c21) & (C2<=c22),arancel_c22,
                                 np.where((C2>c22) & (C2<=c23),arancel_c23,arancel_c23+.04)))
    elif row['TIPO_ASIGANCION'] == 'C3':
        return np.where((C3>0) & (C3<=c31),arancel_c31,
                        np.where((C3>c31) & (C3<=c32),arancel_c32,
                                 np.where((C3>c32) & (C3<=c33),arancel_c33,arancel_c33+.04)))
    elif row['TIPO_ASIGANCION'] == 'C4':
        return np.where((C4>0) & (C4<=c41),arancel_c41,
                        np.where((C4>c41) & (C4<=c42),arancel_c42,
                                 np.where((C4>c42) & (C4<=c43),arancel_c43,arancel_c43+.04)))
    
    else:
        return 0


In [34]:
pagos_brad2['Arancel'] = pagos_brad2.apply(func, axis=1)
pagos_brad2['Facturacion'] = [x*y for x,y in zip(pagos_brad2['MONTO'],pagos_brad2['Arancel'])]
pagos_brad2['Cliente'] = 'Bradesco'

In [35]:
pagos_totales_brad2 = pd.pivot_table(pagos_brad2,index=['Cliente'],values=['Facturacion'],aggfunc=[np.sum],dropna=False)
pagos_totales_brad2 = pd.DataFrame(pagos_totales_brad2.to_records())
pagos_totales_brad2.columns = ['Cliente','Facturacion']
pagos_totales_brad2.fillna(0,inplace=True)
pagos_totales_brad2['FechaFact'] = max(pagos_brad2['FECHA_PAGO'])
pagos_totales_brad2

,Cliente,Facturacion,FechaFact
0,Bradesco,104747.641,2021-01-14


In [36]:
pagos_brad2.head(1)

,NUMERO_TARJETA,FECHA_PAGO,TIPO_ASIGANCION,MONTO,Arancel,Facturacion,Cliente
0,2869000840278754,2021-01-08,C1,500.0,0.17,85.0,Bradesco


In [37]:
pagos_brad_gerencia = pd.pivot_table(pagos_brad2,index=['TIPO_ASIGANCION'],values=['MONTO'],aggfunc=[np.sum])
pagos_brad_gerencia = pd.DataFrame(pagos_brad_gerencia.to_records())
pagos_brad_gerencia.columns = ['Asignacion','Recuperado']
pagos_brad_gerencia.fillna(0,inplace=True)
pagos_brad_gerencia['FechaFact'] = max(pagos_brad2['FECHA_PAGO'])
pagos_brad_gerencia['SiguienteArancel'] = pagos_brad_gerencia.apply(sig_arancel, axis=1)
pagos_brad_gerencia['RestSigArancel'] = pagos_brad_gerencia['SiguienteArancel']-pagos_brad_gerencia['Recuperado']
cols = ['Recuperado','SiguienteArancel','RestSigArancel']
pagos_brad_gerencia[cols] = pagos_brad_gerencia[cols].round(2)
#pagos_brad_gerencia = pagos_brad_gerencia.iloc[:,[0,1,2,4]]
pagos_brad_gerencia.columns = ['Division','Recuperado','FechaFact','SiguienteArancel','RestSigArancel']
pagos_brad_gerencia

,Division,Recuperado,FechaFact,SiguienteArancel,RestSigArancel
0,A,32007.00,2021-01-14,264024,232017
1,B,5047.00,2021-01-14,214969,209922
2,C0 4PV,266056.32,2021-01-14,846436,580380
3,C0 5PV,112172.10,2021-01-14,729630,617458
4,C0 6PV,92226.50,2021-01-14,623524,531298
5,C1,135503.00,2021-01-14,1430989,1.29549e+06
6,C2,109827.84,2021-01-14,549549,439721
7,C3,52510.00,2021-01-14,237647,185137
8,C4,8574.00,2021-01-14,109401,100827


# Credifiel

In [38]:
#Hacemos el consolidado de la asignacion y promesado
consolidado_pagos_cred = pd.merge(pagos_cred2,asignacion_cred,how='left',left_on=['ID'],right_on=['CREDITO'])
consolidado_pagos_cred.drop('CREDITO',1,inplace=True)
consolidado_pagos_cred['ARANCEL'] = np.where(consolidado_pagos_cred['ARANCEL'].isna(),np.mean(consolidado_pagos_cred['ARANCEL']),consolidado_pagos_cred['ARANCEL'])
consolidado_pagos_cred['FACTURACION'] = consolidado_pagos_cred['IMPORTE']*consolidado_pagos_cred['ARANCEL']
consolidado_pagos_cred['CLIENTE'] = 'Credifiel'
consolidado_pagos_cred.head(1)

,ID,FECHA,IMPORTE,ARANCEL,FACTURACION,CLIENTE
0,76245,2021-01-11,30000.0,0.202414,6072.413793,Credifiel


In [39]:
pagos_totales_cred = pd.pivot_table(consolidado_pagos_cred,index=['CLIENTE'],values=['FACTURACION'],aggfunc=[np.sum],dropna=False)
pagos_totales_cred = pd.DataFrame(pagos_totales_cred.to_records())
#pagos_totales_cred = pagos_totales_cred.iloc[:,[0,1,4,3]]
pagos_totales_cred.columns = ['Cliente','Facturacion']
pagos_totales_cred.fillna(0,inplace=True)
#pagos_totales_cred[['SumaPagos','SumaFacturacion']] = pagos_totales_cred[['SumaPagos','SumaFacturacion']].applymap("${0:,.2f}".format)
pagos_totales_cred['FechaFact'] = max(consolidado_pagos_cred['FECHA'])
pagos_totales_cred

,Cliente,Facturacion,FechaFact
0,Credifiel,29887.543862,2021-01-13


In [40]:
pagos_cred_gerencia = pd.pivot_table(consolidado_pagos_cred,index=['CLIENTE'],values=['IMPORTE'],aggfunc=np.sum)
pagos_cred_gerencia = pd.DataFrame(pagos_cred_gerencia.to_records())
pagos_cred_gerencia[' FechaFact'] = max(consolidado_pagos_cred['FECHA'])
pagos_cred_gerencia.columns = ['Division','Recuperado','FechaFact']
pagos_cred_gerencia

,Division,Recuperado,FechaFact
0,Credifiel,174325.96,2021-01-13


# Banco Azteca

In [41]:
# pagos_baz = pd.concat([pagos_baz_26,pagos_baz_recovery],ignore_index=True)
# pagos_baz = pagos_baz.loc[pagos_baz['CARGO AUTOMATICO']=='NO']
# pagos_baz['Recuperado'] = pagos_baz['RECUPERACION DE CAPITAL']+pagos_baz['RECUPERACION DE MORATORIOS']
# pagos_baz['Cliente'] = 'Banco Azteca'

In [42]:
# pagos_totales_baz = pd.pivot_table(fact_baz,index=['Cliente'],values=['Total Recuperado','Total Comisión'],aggfunc=[np.sum],dropna=False)
# pagos_totales_baz = pd.DataFrame(pagos_totales_baz.to_records())
# pagos_totales_baz.columns = ['Cliente','Recuperado','SumaPagos']
# porcentaje_prom_arancel_baz = pagos_totales_baz['Recuperado']/pagos_totales_baz['SumaPagos']
# porcentaje_prom_arancel_baz

In [43]:
# pagos_baz['Facturacion'] = [x*porcentaje_prom_arancel_baz for x in pagos_baz['Recuperado']]
# pagos_fact_baz = pd.pivot_table(pagos_baz,index=['Cliente'],values=['Facturacion'],aggfunc=[np.sum])
# pagos_fact_baz = pd.DataFrame(pagos_fact_baz.to_records())
# pagos_fact_baz.columns = ['Cliente','Facturacion']
# pagos_fact_baz['FechaFact'] = max(pagos_baz['DIA'])
# pagos_fact_baz

# Union Naucalpan

In [44]:
pdList = [pagos_totales_liv2,pagos_totales_brad2,pagos_totales_cred]
final_naucalpan = pd.concat(pdList,ignore_index=True)
final_naucalpan = pd.merge(final_naucalpan,meta_naucalpan,on=['Cliente'],how='left')
final_naucalpan['Avance%'] = final_naucalpan['Facturacion']/final_naucalpan['Meta']
final_naucalpan.insert(loc=0,column='Sucursal',value='Naucalpan')
final_naucalpan

,Sucursal,Cliente,Facturacion,FechaFact,Meta,Avance%
0,Naucalpan,Liverpool,100314.390000,2021-01-14,300000,0.334381
1,Naucalpan,Liverpool Cof,71746.680000,2021-01-14,300000,0.239156
2,Naucalpan,Liverpool Nacional,108353.200000,2021-01-14,550000,0.197006
3,Naucalpan,Bradesco,104747.641000,2021-01-14,750000,0.139664
4,Naucalpan,Credifiel,29887.543862,2021-01-13,100000,0.298875


# Guatemala

In [45]:
final_guatemala = pd.DataFrame([['Avon G',66727,250000],['Arabela G',808,5100],['Banco Azteca G',4542,52000],['Banco Credomatic',2222,17000],['Belcorp',2010,5000],['Banco Azteca R',27375,115000]])
final_guatemala.columns = ['Cliente','Facturacion','Meta']
final_guatemala.insert(loc=0,column='Sucursal',value='Guatemala')

In [46]:
final_guatemala['Facturacion'] = final_guatemala['Facturacion']*quetzal
final_guatemala['Meta'] = final_guatemala['Meta']*quetzal
final_guatemala['FechaFact'] = ayer

# Panama

In [47]:
final_panama = pd.DataFrame([['Cable Onda',780,2800],['Jamar',1150,8000],['Banco Azteca',1745,4000],['Adel Panama',383,3000],['Cable Onda Act',357,2000],['Banco Azteca Act',12024,23500]])
final_panama.columns = ['Cliente','Facturacion','Meta']
final_panama.insert(loc=0,column='Sucursal',value='Panama')

In [48]:
final_panama['Facturacion'] = final_panama['Facturacion']*dolar
final_panama['Meta'] = final_panama['Meta']*dolar
final_panama['FechaFact'] = ayer

# Union final

In [49]:
final_mexico = pd.concat([final_naucalpan,final_torreon],ignore_index=True)
final_mexico = pd.concat([final_mexico,final_guatemala],ignore_index=True)
final_mexico = pd.concat([final_mexico,final_panama],ignore_index=True)
final_mexico['Avance%'] = final_mexico['Facturacion']/final_mexico['Meta']
final_mexico

,Sucursal,Cliente,Facturacion,FechaFact,Meta,Avance%
0,Naucalpan,Liverpool,100314.390000,2021-01-14,300000.0,0.334381
1,Naucalpan,Liverpool Cof,71746.680000,2021-01-14,300000.0,0.239156
2,Naucalpan,Liverpool Nacional,108353.200000,2021-01-14,550000.0,0.197006
3,Naucalpan,Bradesco,104747.641000,2021-01-14,750000.0,0.139664
4,Naucalpan,Credifiel,29887.543862,2021-01-13,100000.0,0.298875
5,Torreon,Arabela,47829.293600,2021-01-12,95000.0,0.503466
6,Torreon,Invex_REL,67912.764000,2021-01-11,470000.0,0.144495
7,Torreon,Invex_RLA,78187.004000,2021-01-11,479000.0,0.163230
8,Torreon,Invex_RLN,58453.902000,2021-01-11,270000.0,0.216496
9,Torreon,Oriflame,34482.005000,2021-01-11,65000.0,0.530492


In [50]:
final_mexico['MetaDia'] = ((final_mexico['FechaFact'].dt.day/30)*final_mexico['Meta']).round()
final_mexico['Semaforo'] = np.where(final_mexico['MetaDia']<=final_mexico['Facturacion'],'verde',
                                    np.where((final_mexico['MetaDia']*.75<=final_mexico['Facturacion']) &                                                                                    (final_mexico['MetaDia']>final_mexico['Facturacion']),'amarillo','rojo'))
final_mexico
fecha_fact = datetime.strftime(max(final_mexico['FechaFact']),'%d-%m-%Y')

,Sucursal,Cliente,Facturacion,FechaFact,Meta,Avance%,MetaDia,Semaforo
0,Naucalpan,Liverpool,100314.390000,2021-01-14,300000.0,0.334381,140000.0,rojo
1,Naucalpan,Liverpool Cof,71746.680000,2021-01-14,300000.0,0.239156,140000.0,rojo
2,Naucalpan,Liverpool Nacional,108353.200000,2021-01-14,550000.0,0.197006,256667.0,rojo
3,Naucalpan,Bradesco,104747.641000,2021-01-14,750000.0,0.139664,350000.0,rojo
4,Naucalpan,Credifiel,29887.543862,2021-01-13,100000.0,0.298875,43333.0,rojo
5,Torreon,Arabela,47829.293600,2021-01-12,95000.0,0.503466,38000.0,verde
6,Torreon,Invex_REL,67912.764000,2021-01-11,470000.0,0.144495,172333.0,rojo
7,Torreon,Invex_RLA,78187.004000,2021-01-11,479000.0,0.163230,175633.0,rojo
8,Torreon,Invex_RLN,58453.902000,2021-01-11,270000.0,0.216496,99000.0,rojo
9,Torreon,Oriflame,34482.005000,2021-01-11,65000.0,0.530492,23833.0,verde


In [51]:
final_sucursal = pd.pivot_table(final_mexico,index=['Sucursal'],values=['Meta','Facturacion','FechaFact'],aggfunc=[np.sum,np.max])
final_sucursal = pd.DataFrame(final_sucursal.to_records())
final_sucursal = final_sucursal.iloc[:,[0,1,2,4]]
final_sucursal.columns = ['Sucursal','Facturacion','Meta','FechaFact']
final_sucursal['Avance'] = final_sucursal['Facturacion']/final_sucursal['Meta']
final_sucursal['MetaDia'] = ((final_sucursal['FechaFact'].dt.day/30)*final_sucursal['Meta']).round()
final_sucursal['Semaforo'] = np.where(final_sucursal['MetaDia']<=final_sucursal['Facturacion'],'verde',
                                    np.where((final_sucursal['MetaDia']*.75<=final_sucursal['Facturacion']) &(final_sucursal['MetaDia']>final_sucursal['Facturacion']),'amarillo','rojo'))
final_sucursal

,Sucursal,Facturacion,Meta,FechaFact,Avance,MetaDia,Semaforo
0,Guatemala,263357.360000,1128014.0,2021-01-14,0.233470,526407.0,rojo
1,Naucalpan,415049.454862,2000000.0,2021-01-14,0.207525,933333.0,rojo
2,Panama,325492.200000,857340.0,2021-01-14,0.379654,400092.0,amarillo
3,Torreon,286864.968600,1379000.0,2021-01-12,0.208024,551600.0,rojo


In [52]:
final_bradlivcred = pd.concat([pagos_liv_gerencia,pagos_brad_gerencia],ignore_index=True)
final_bradlivcred = pd.concat([final_bradlivcred,pagos_cred_gerencia],ignore_index=True)
final_bradlivcred

,Division,Recuperado,FechaFact,SiguienteArancel,RestSigArancel
0,DKN00REM,302061.00,2021-01-14,1071428.6,769368
1,DKN02REM,270882.99,2021-01-14,1625000.0,1.35412e+06
2,DKN04REM,143493.37,2021-01-14,600000.0,456507
3,SA001REM,100220.98,2021-01-14,2000000.0,1.89978e+06
4,SA002REM,4401.00,2021-01-14,2187500.0,2.1831e+06
5,A,32007.00,2021-01-14,264024,232017
6,B,5047.00,2021-01-14,214969,209922
7,C0 4PV,266056.32,2021-01-14,846436,580380
8,C0 5PV,112172.10,2021-01-14,729630,617458
9,C0 6PV,92226.50,2021-01-14,623524,531298


In [53]:
reporte_pagos_nauc = pd.read_csv(''+ruta_pagos+'/'+mes2+'/Reportes diarios/Reporte sucursal/Reporte sucursal '+today2+'.csv')

In [54]:
final_bradlivcred2 = final_bradlivcred.iloc[:,[0,3,4]]
final_bradlivcred3 = pd.merge(reporte_pagos_nauc,final_bradlivcred2,on=['Division'],how='left')
cols = ['SumaFacturacion','TendenciaCierre1']
final_bradlivcred3[cols] = final_bradlivcred3[cols].round(2)
final_bradlivcred3

,Pais,Sucursal,Cliente,Division,NumeroPagos,SumaPagos,SumaFacturacion,FechaFact,NumeroPromesas,SumaPromesas,PromesasRestantes,TendenciaCierre1,RecuperadoNoPromesa,PagosNoPromesa,SiguienteArancel,RestSigArancel
0,Mexico,Naucalpan,Liverpool,DKN00REM,73,302061.00,84577.08,2021-01-14,131,539900.87,196262.00,181236.60,59503.00,18.0,1071428.6,769368
1,Mexico,Naucalpan,Liverpool,DKN02REM,65,270882.99,108353.20,2021-01-14,109,523466.44,200205.00,232185.42,58605.99,31.0,1625000.0,1.35412e+06
2,Mexico,Naucalpan,Liverpool,DKN04REM,35,143493.37,71746.68,2021-01-14,58,195982.37,68934.00,153742.90,26263.00,5.0,600000.0,456507
3,Mexico,Naucalpan,Liverpool,SA001REM,67,100220.98,15033.15,2021-01-14,120,201336.24,110193.10,32213.89,20608.83,18.0,2000000.0,1.89978e+06
4,Mexico,Naucalpan,Liverpool,SA002REM,5,4401.00,704.16,2021-01-14,10,21957.00,5092.00,1508.91,2411.00,3.0,2187500.0,2.1831e+06
5,Mexico,Naucalpan,Bradesco,A,14,32007.00,8641.89,2021-01-14,27,51654.00,5000.00,18518.34,3720.00,1.0,264024,232017
6,Mexico,Naucalpan,Bradesco,B,4,5047.00,1261.75,2021-01-14,6,10201.00,404.00,2703.75,1500.00,1.0,214969,209922
7,Mexico,Naucalpan,Bradesco,C0 4PV,183,266056.32,18623.94,2021-01-14,251,397336.09,63908.00,39908.45,98095.32,79.0,846436,580380
8,Mexico,Naucalpan,Bradesco,C0 5PV,87,112172.10,10095.49,2021-01-14,114,192308.00,32595.00,21633.19,49070.08,41.0,729630,617458
9,Mexico,Naucalpan,Bradesco,C0 6PV,64,92226.50,9222.65,2021-01-14,91,171749.00,20352.00,19762.82,35167.50,31.0,623524,531298


In [55]:
writer = pd.ExcelWriter(''+ruta_guardar+'/Avance Pagos '+fecha_fact+'.xlsx',engine='xlsxwriter')
final_sucursal.to_excel(writer,'FactSucursal',index=False,header=True)
final_mexico.to_excel(writer,'FactCliente',index=False,header=True)
final_bradlivcred3.to_excel(writer,'GerEnrique',index=False,header=True)

writer.save()